<h1>Roadmap</h1>
<p></p>

<ol>
    <li style="color:green">Explore a website, create and test separate parsing scripts for different levels of parsing.</li>
    <li style="color:green">Create parsing_module.py containing parsing functions</li>
    <li style="color:green">Define parsing schema</li>
    <li style="color:green">Define the structure of a database</li>
    <li style="color:green">Set up the database on a machine</li>
    <li style="color:green">Parse all product links into Postgres db</li>
    <li style="color:green">Extend product_links_table to accomodate product data (add celery to create parsing intervals)</li>
    <li style="color:orange">Parse each product page into db</li>
    <li>Create API for data access</li>
</ol>

<h3>Define Parsing Schema</h3>

<p>I would like to parse all product links in the first parsing stage and then use this links to parse product info on the second stage.</p>
<img src="screenshots/parsed_dataframe.png" align="left" width="600">

<p>The website contais <b>1224</b> subcategory_lvl_2, each subcategory has a link to its products page. I've already made a script that extracts all product links from single subcategory_lvl_2. The shop claims to have about <b>75 000 products </b> in catalog in total.</p>

<p>It seems like a good idea to parse 75 000 product links in the first place because this will enable quality parsing of separate products on the second stage. Especially considering the fact that we have to extract and store similar products.</p>

<h3>Define Database Structure</h3>
<p>I've discovered that tree-like database sctructure seems like the best choice for our case.</p>
<img src="screenshots/tree_schematic.png" align="left" width="600">

<p>From https://kite.com/blog/python/sqlalchemy :</p>
<p>There are a variety of well-known approaches for storing trees in a relational database. A common technique is using the <i><b>materialized path pattern</b></i>, in which each node keeps a record of the path to reach it from the root of the tree. This approach allows for fast inserts and fast queries, but moving an existing node to a different tree can be slow and expensive, as you have to rewrite the paths on any descendants of that node. </p>
<p>If you happen to be using Postgres as your database – you’re in luck! Postgres actually offers a custom data type called <i><b>LTree</i></b> specifically designed to record materialized paths for representing trees.</p>

<h3>Use intermediate table?</h3>
<p>I have collected all product links intor single table in postgres db. The table contains <b>24092 entries</b> and has following columns:</p>
<ul>
    <li>id</li>
    <li>url</li>
    <li>category</li>
    <li>subcat_lvl1</li>
    <li>subcat_lvl2</li>
</ul>

<img src="screenshots/parsed_table.png" align="left" width="600">


<p>Implementing tree-based database does not seem completely transparent. So it seems like a good idea to extend an existing table with product attributes and the parse each product into this table.</p>
<ul>
    <li>product_name <i>VARCHAR</i></li>
    <li>product_price <i>NUMERIC (5, 2)</i></li>
    <li>product_description <i>VARCHAR</i></li>
    <li>product_characteristics <i>VARCHAR</i></li>
    <li style="color:red">similar_products <b>(could not get similar products (ajax call))</b> <i>VARCHAR for now</i></li>
    <li>product_image_link <i>VARCHAR</i></li>
    <li>product_is_hit <i>BOOLEAN</i></li>
    <li>is_parsed <i>TIMESTAMP</i></li>
</ul>

<p>Getting similar products out of plane html does not seem possible, so i will leave similar column empty for now and fill the intermediate table with product information.</p>

<h3>Extend existing product links table</h3>
<p>I have extended existing table with new columns according to task requirements</p>

<img src="screenshots/product_table.png" align="left" width="500">


<h3>What's next:</h3>
<p>1. Iterate over product links and collect product data from each page using get_product_parameters function in parsing_module.py. </p>
<p>2. When parsing is done: create tree-type database out of single preliminary denormalized table. </p>
<p>3. Create API for access. </p>

<h3>Exectution time estimation</h3>
<p>I've tried to parse 300 product pages and it too 4 minutes 51 secods. So without adding multiprocessing the whole process of parsing each product entry will take about 6,5 hours.</p>




<h3>Collect product data into extended products_all table</h3>
<p>I've tried to parse 500 product pages and it took 8 minutes 46 secods. So without adding multiprocessing the whole process of parsing each product entry will take about 6,5 hours.</p>
<img src="screenshots/parse_product_info_into_extended_db.png" align="left" width="600">



<p>I've also tested retrieving parsed product data from db by id.</p>
<img src="screenshots/parsed_product.png" align="left" width="600">